# Adding pixels above and below center line

**Methods:**
>1. Load data
>2. Calculate pixels above and below
>3. Export to processed dataset

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.svm as skl_svm
import sklearn.cross_validation as skl_cv
import seaborn as sns
sns.set_context(context = 'notebook', font_scale=1.5)
import os
import sys
import cv2

base_path = '/home/lundi/Python/MNIST/'
sys.path.append(base_path + '/libraries/')

import MNIST_data_processor as mdp

MNIST_data_processor = mdp.MNIST_data_processor()

from matplotlib.patches import Ellipse

## 1. Load data

In [2]:
X, y = MNIST_data_processor.load_subset_data()

## 2. Calculate pixels above and below

In [3]:
def calculate_center_of_number(digit_data):
    digit_data_reshape = pd.DataFrame(digit_data.reshape(28,28))
    pixel_indexes = digit_data_reshape.max(axis=1).replace(0, np.nan).dropna()
    #print pixel_indexes
    mid_index = int(1.0 * pixel_indexes.shape[0] / 2.0)
    return pixel_indexes.index[mid_index]

def get_pixels_above_and_below(digit_data, mid_line):
    digit_data_reshape = pd.DataFrame(digit_data.reshape(28,28))
    top_region = digit_data_reshape.ix[0:mid_line-1]
    bottom_region = digit_data_reshape.ix[mid_line:]
    
    return top_region.sum().sum(), bottom_region.sum().sum()

results = []
for index, current_digit_data in X.iterrows():
    mid_line = calculate_center_of_number(current_digit_data)
    top_pixels, bottom_pixels = get_pixels_above_and_below(current_digit_data, mid_line)
    results.append([y[index], top_pixels, bottom_pixels])
    
results_df = pd.DataFrame(results, columns = ['label', 'top','bottom'])

## 3. Export to processed dataset

In [5]:
results_df.reset_index().rename(columns = {'index': 'datum_index'}).to_csv('./../../data/generated_features/pixels_above_and_below_center.csv', index=False)